In [36]:
from bs4 import BeautifulSoup
import requests

from datetime import datetime
from dateutil.parser import parse
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

from pprint import pprint
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc 
import random
from tqdm import tqdm
import pandas as pd

In [37]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

In [38]:
# reading excel file
df = pd.read_excel('C:/Users/tobias.sion/Desktop/GIT 5 Sept 2023.xlsx') 
parse_bl = df['CNC'].tolist()
bl_list = []
for i in parse_bl:
    bl_list.append(str(i))
bl_list = [x.replace(' ', '') for x in bl_list]
bl_list = [x for x in bl_list if x != 'nan']

In [39]:
bl_list

['ARM0296363',
 'ARM0302106',
 'ARM0300326',
 'ARM0300876',
 'ARM0300209',
 'ARM0300221',
 'ARM0301801',
 'ARM0303785',
 'ARM0303661',
 'ARM0301769',
 'ARM0301802',
 'ARM0301958',
 'ARM0302330',
 'ARM0302741',
 'ARM0302585',
 'ARM0302772',
 'ARM0303044',
 'ARM0303415',
 'ARM0303224',
 'ARM0305556',
 'ARM0305653',
 'ARM0305513',
 'ARM0306500']

In [40]:
from datetime import datetime
def parse_date(date_str):
    return datetime.strptime(date_str, '%d-%b-%Y')

In [41]:
hasil_akhir = []

In [42]:
hasil_akhir = []
# web scripting
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome()
driver.get("https://www.cnc-line.com/ebusiness/tracking/search")

# search box
search_box = driver.find_element(By. XPATH, '/html/body/div[2]/main/section/div/div/form[3]/fieldset/div/div[1]/span[1]/input[2]')
search_box.send_keys(bl_list[random.randrange(0,len(bl_list))])

# click search
time.sleep(1)
search_button = driver.find_element(By.XPATH, '/html/body/div[2]/main/section/div/div/form[3]/fieldset/div/div[2]/button')
time.sleep(1)
search_button.click()

for q, bls in enumerate(tqdm(bl_list)):
    try:
        time.sleep(random.randrange(2,5))
        # masukin BL baru
        search_box2 = driver.find_element(By. XPATH, '/html/body/div[2]/main/section[1]/div/div/form[3]/fieldset/div/div[1]/span[1]/input[2]')
        # driver.execute_script("window.scrollTo(100,document.body.scrollHeight);")
        search_box2.clear()
        search_box2.send_keys(bls)
        time.sleep(1)
        search_button = driver.find_element(By.XPATH, '/html/body/div[2]/main/section[1]/div/div/form[3]/fieldset/div/div[2]/button')
        time.sleep(1)
        search_button.click()
            

        time.sleep(1)
        # taking data from web 
        soup = BeautifulSoup(driver.page_source, 'lxml')

        # finding containers in bl
        containers_in_bl = soup.find_all('dl',{'class','container-ref'})
        list_of_containers = []
        for tag in containers_in_bl:
            for e, f  in enumerate(tag.find_all('span')):
                if len(f.text) == 11:
                    list_of_containers.append(f.text)

        if len(list_of_containers)>1:
            print(bls, 'consist of ', len(list_of_containers),' containers' )
            # more
            # driver.execute_script("window.scrollTo(100,document.body.scrollHeight);")
            more_button= WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/main/section[2]/div/div/ul/li[1]/article/section[2]/div[1]/div/label'))) 
            more_button.click()

            time.sleep(3)
            soup1 = BeautifulSoup(driver.page_source, 'lxml')

            # finding milestone and expected data
            data_milestone = soup1.find_all('tr',{'class','k-master-row done'}) + soup1.find_all('tr',{'class','k-alt k-master-row done'}) + soup1.find_all('tr',{'class','k-alt k-master-row current'}) + soup1.find_all('tr',{'class','k-master-row current'})
            data_expected = soup1.find_all('tr',{'class','k-master-row inactivek-alt'}) + soup1.find_all('tr',{'class','k-master-row inactive'})
            headline = soup1.find_all('strong')

            # making a list of milestones
            list_of_milestone_date = []
            list_of_milestone_movement = []
            list_of_milestone_cities = []
            for tag in data_milestone:
                for a, b  in enumerate(tag.find_all('span',{'class','calendar'})):
                    list_of_milestone_date.append(b.text[-11:])
                for c, d  in enumerate(tag.find_all('span',{'class','capsule'})):
                    list_of_milestone_movement.append(d.text)
                for e, f  in enumerate(tag.find_all('div',{'class','location row js-bubble'})):
                    list_of_milestone_cities.append(f.text[:-16])

            # making a list of expected dict
            list_of_expected_date = []
            list_of_expected_movement = []
            list_of_expected_cities = []
            for tag in data_expected:
                for g, h  in enumerate(tag.find_all('span',{'class','calendar'})):
                    list_of_expected_date.append(h.text[-11:])
                for i, j  in enumerate(tag.find_all('span',{'class','capsule'})):
                    list_of_expected_movement.append(j.text)
                for k, l  in enumerate(tag.find_all('div',{'class','location row js-bubble'})):
                    list_of_expected_cities.append(l.text[:-16])

            # Apending data from list to current dict
            current_dict = {}
            for m, milestone in enumerate(sorted(list_of_milestone_date)):
                if list_of_milestone_cities[m] == headline[1].text[:-5] or list_of_milestone_cities[m] == headline[2].text[:-5]:
                    case_milestone = {list_of_milestone_movement[m] +' '+ list_of_milestone_cities[m] :list_of_milestone_date[m]}
                    current_dict.update(case_milestone)

            for e, expected in enumerate(sorted(list_of_expected_date)):
                if list_of_expected_cities[e] == headline[2].text[:-5] or list_of_expected_cities[e] == headline[1].text[:-5]:
                    case_expected = {"EXPECTED " + list_of_expected_movement[e] + ' ' +  list_of_expected_cities[e]:list_of_expected_date[e]}
                    current_dict.update(case_expected)

            # sorting dictionary using date
            current_dict = dict(sorted(current_dict.items(), key=lambda item: parse_date(item[1])))

            # cities from to
            cities = soup1.find_all('ul',{'class','timeline--items'})

            for x, container in enumerate(list_of_containers):
                appending_dict = current_dict.copy()
                for tag in cities:
                    for cc, city in enumerate(tag.find_all('div',{'class':'timeline--item-description'})):
                        case_city = {city.text.replace("\n","")[:3]:city.text.replace("\n","")[3:-5]}
                        appending_dict.update(case_city)
                appending_dict.update({"Container Number":container})
                appending_dict.update({"Bill of Lading number":bls})
                hasil_akhir.append(appending_dict)
            print(bls, container, " DONE")
            time.sleep(2)
        else:
            print(bls, 'consist of 1 containers' )
            # souping
            time.sleep(2)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')

            current_dict = {}
            # finding milestone and expected data
            data_milestone = soup2.find_all('tr',{'class','k-master-row done'}) + soup2.find_all('tr',{'class','k-alt k-master-row done'}) + soup2.find_all('tr',{'class','k-alt k-master-row current'}) + soup2.find_all('tr',{'class','k-master-row current'})
            data_expected = soup2.find_all('tr',{'class','k-alt k-master-row inactive'}) + soup2.find_all('tr',{'class','k-master-row inactive'})
            headline = soup2.find_all('strong')

            # making a list of milestones
            list_of_milestone_date = []
            list_of_milestone_movement = []
            list_of_milestone_cities = []
            for tag in data_milestone:
                for a, b  in enumerate(tag.find_all('span',{'class','calendar'})):
                    list_of_milestone_date.append(b.text[-11:])
                for c, d  in enumerate(tag.find_all('span',{'class','capsule'})):
                    list_of_milestone_movement.append(d.text)
                for e, f  in enumerate(tag.find_all('div',{'class','location row js-bubble'})):
                    list_of_milestone_cities.append(f.text[:-16])

            # making a list od expected dict
            list_of_expected_date = []
            list_of_expected_movement = []
            list_of_expected_cities = []
            for tag in data_expected:
                for g, h  in enumerate(tag.find_all('span',{'class','calendar'})):
                    list_of_expected_date.append(h.text[-11:])
                for i, j  in enumerate(tag.find_all('span',{'class','capsule'})):
                    list_of_expected_movement.append(j.text)
                for k, l  in enumerate(tag.find_all('div',{'class','location row js-bubble'})):
                    list_of_expected_cities.append(l.text[:-16])

            # Apending data from list to current dict
            current_dict = {}
            for m, milestone in enumerate(sorted(list_of_milestone_date)):
                if list_of_milestone_cities[m] == headline[2].text[:-5] or list_of_milestone_cities[m] == headline[3].text[:-5]:
                    case_milestone = {list_of_milestone_movement[m] +' '+ list_of_milestone_cities[m] :list_of_milestone_date[m]}
                    current_dict.update(case_milestone)
                    

            for e, expected in enumerate(sorted(list_of_expected_date)):
                if list_of_expected_cities[e] == headline[3].text[:-5] or list_of_expected_cities[e] == headline[2].text[:-5]:
                    case_expected = {"EXPECTED " + list_of_expected_movement[e] + ' ' +  list_of_expected_cities[e]:list_of_expected_date[e]}
                    current_dict.update(case_expected)

            current_dict_fix = dict(sorted(current_dict.items(), key=lambda item: parse_date(item[1])))
            current_dict_fix.update({"Bill of Lading number":bls})
            current_dict_fix.update({"Container Number":headline[0].text})
            current_dict_fix.update({"POL":headline[2].text[:-5]})
            current_dict_fix.update({"POD":headline[3].text[:-5]})

            hasil_akhir.append(current_dict_fix)
            print(bls, headline[0].text, " DONE")
            time.sleep(2)
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)

  0%|          | 0/23 [00:00<?, ?it/s]

ARM0296363 consist of  5  containers
ARM0296363 CMAU8827659  DONE


  4%|▍         | 1/23 [00:12<04:28, 12.20s/it]

ARM0302106 consist of  2  containers
ARM0302106 CMAU8582181  DONE


  9%|▊         | 2/23 [00:25<04:27, 12.75s/it]

...

 91%|█████████▏| 21/23 [04:33<00:25, 12.50s/it]

ARM0305513 consist of  3  containers
ARM0305513 TCKU6354975  DONE


 96%|█████████▌| 22/23 [04:47<00:12, 12.81s/it]

ARM0306500 consist of  13  containers
ARM0306500 BHCU4961587  DONE


100%|██████████| 23/23 [05:00<00:00, 13.08s/it]


In [43]:
hasil_akhir2 = hasil_akhir

In [44]:
hasil_akhir2

[{'Empty to shipper JAKARTA': '17-MAY-2023',
  'Ready to be loaded JAKARTA': '21-MAY-2023',
  'Loaded on board JAKARTA': '25-MAY-2023',
  'Vessel Departure JAKARTA': '26-MAY-2023',
  'Vessel Arrival MANILA': '02-JUN-2023',
  'Discharged MANILA': '03-JUN-2023',
  'Container to consignee MANILA': '06-SEP-2023',
  'POL': 'JAKARTA',
  'POD': 'MANILA',
  'Container Number': 'TCKU6232468',
  'Bill of Lading number': 'ARM0296363'},
 {'Empty to shipper JAKARTA': '17-MAY-2023',
  'Ready to be loaded JAKARTA': '21-MAY-2023',
  'Loaded on board JAKARTA': '25-MAY-2023',
  'Vessel Departure JAKARTA': '26-MAY-2023',
  'Vessel Arrival MANILA': '02-JUN-2023',
  'Discharged MANILA': '03-JUN-2023',
  'Container to consignee MANILA': '06-SEP-2023',
  'POL': 'JAKARTA',
  'POD': 'MANILA',
  'Container Number': 'CMAU8535106',
  'Bill of Lading number': 'ARM0296363'},
 {'Empty to shipper JAKARTA': '17-MAY-2023',
  'Ready to be loaded JAKARTA': '21-MAY-2023',
  'Loaded on board JAKARTA': '25-MAY-2023',
  'Ves

In [45]:
list_of_dict_fix = []
for filter_dict in hasil_akhir2:
    replacement_mapping = {
        filter_dict["POL"]: 'Origin',
        filter_dict["POD"]: 'Destination'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

    list_of_dict_fix.append(updated_dict)

In [46]:
list_of_dict_fix

[{'Empty to shipper Origin': '17-MAY-2023',
  'Ready to be loaded Origin': '21-MAY-2023',
  'Loaded on board Origin': '25-MAY-2023',
  'Vessel Departure Origin': '26-MAY-2023',
  'Vessel Arrival Destination': '02-JUN-2023',
  'Discharged Destination': '03-JUN-2023',
  'Container to consignee Destination': '06-SEP-2023',
  'POL': 'JAKARTA',
  'POD': 'MANILA',
  'Container Number': 'TCKU6232468',
  'Bill of Lading number': 'ARM0296363'},
 {'Empty to shipper Origin': '17-MAY-2023',
  'Ready to be loaded Origin': '21-MAY-2023',
  'Loaded on board Origin': '25-MAY-2023',
  'Vessel Departure Origin': '26-MAY-2023',
  'Vessel Arrival Destination': '02-JUN-2023',
  'Discharged Destination': '03-JUN-2023',
  'Container to consignee Destination': '06-SEP-2023',
  'POL': 'JAKARTA',
  'POD': 'MANILA',
  'Container Number': 'CMAU8535106',
  'Bill of Lading number': 'ARM0296363'},
 {'Empty to shipper Origin': '17-MAY-2023',
  'Ready to be loaded Origin': '21-MAY-2023',
  'Loaded on board Origin': '2

In [47]:
list_of_dict_fix2 = []
for filter_dict in list_of_dict_fix:
    replacement_mapping = {
        "Vessel Departure Origin": 'ATD',
        "Discharged Destination": 'ATA',
        "EXPECTED Vessel Arrival Destination" : "ETD",
        "Container to consignee Destination": 'Container Release',
        "Empty in depot Destination" : 'Container Return'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

    list_of_dict_fix2.append(updated_dict)

In [48]:
list_of_dict_fix2

[{'Empty to shipper Origin': '17-MAY-2023',
  'Ready to be loaded Origin': '21-MAY-2023',
  'Loaded on board Origin': '25-MAY-2023',
  'ATD': '26-MAY-2023',
  'Vessel Arrival Destination': '02-JUN-2023',
  'ATA': '03-JUN-2023',
  'Container Release': '06-SEP-2023',
  'POL': 'JAKARTA',
  'POD': 'MANILA',
  'Container Number': 'TCKU6232468',
  'Bill of Lading number': 'ARM0296363'},
 {'Empty to shipper Origin': '17-MAY-2023',
  'Ready to be loaded Origin': '21-MAY-2023',
  'Loaded on board Origin': '25-MAY-2023',
  'ATD': '26-MAY-2023',
  'Vessel Arrival Destination': '02-JUN-2023',
  'ATA': '03-JUN-2023',
  'Container Release': '06-SEP-2023',
  'POL': 'JAKARTA',
  'POD': 'MANILA',
  'Container Number': 'CMAU8535106',
  'Bill of Lading number': 'ARM0296363'},
 {'Empty to shipper Origin': '17-MAY-2023',
  'Ready to be loaded Origin': '21-MAY-2023',
  'Loaded on board Origin': '25-MAY-2023',
  'ATD': '26-MAY-2023',
  'Vessel Arrival Destination': '02-JUN-2023',
  'ATA': '03-JUN-2023',
  'C

In [49]:
import datetime
import pandas as pd
d_today = str(datetime.date.today())
df = pd.DataFrame(list_of_dict_fix2)

excel_file_path = 'CNC.xlsx'

df.to_excel(excel_file_path, index=False)